# Spotify dataset analysis

O Spotify possui uma Web API e essa API possui uma tradução para Python chamada Spotipy. Ela permite buscar informações de músicas, playlists artistas. Dentre as informações disponíveis, existem as chamadas *audio features*, que são métricas musicais do tipo `danceability`, `loudness` e `instrumentalness`, que podem ser importantes para modelos de recomendação.

Os dados foram capturados utilizando a API. A abordagem foi a seguinte:
1. Iniciamos com um conjunto grande de usuários do Last.fm, que permite a busca em rede (usuários e seus seguidores);
2. Selecionamos um subconjunto de 1000 desses usuários que também possuem conta no Spotify;
3. Selecionamos suas playlists públicas;
4. Selecionamos as músicas dessas playlists;
5. We selected the audio features and the artists from these tracks.

It's important to note that songs don't have genres using this API. Who has genre is the artist of the track. So it's necessary to gather the artists of the tracks.

At the end, we have:
- Users **tracks** dataset;
- List of **users**;
- **Playlists** of the tracks;
- **Audio features** of the tracks;
- **Artists** of the tracks.

We now make some exploration in order to get known the data and have insights for the recommendation models.

In [1]:
import pandas as pd
import glob

In [2]:
import numpy as np

In [3]:
# Necessary to improve exploration
pd.set_option('display.max_columns', None)

## Initial exploration: datasets

We have many datasets, and we wanna know its variables. Let's sample them.

### Tracks dataset

In [4]:
tracks = pd.read_pickle('../data/sp_tracks_ready_999.pkl')
tracks.sample()

,added_at,added_by,is_local,playlist_id,available_markets,disc_number,duration_ms,explicit,id,name,popularity,track_number,album_type,album_available_markets,album_id,album_name,album_release_date,artists_ids,artists_names,album_artists_ids,album_artists_names
2375,1970-01-01 00:00:00+00:00,,False,0tKjSLKwT7TB5gL3wuK9QW,[],1.0,193506.0,False,6p1giQj176BzBYmxMDuxNj,I Know What I Know - Remastered Album Version,0.0,3.0,album,[],5u8rXBddedO6UtjNpHWRyJ,Graceland (US Release),1986-08-12,[2CvCyf1gEVhI0mX6aFXmVI],[Paul Simon],[2CvCyf1gEVhI0mX6aFXmVI],[Paul Simon]


**Tracks** dataset is big enough to make we split it in many files. So we did it. At now we have 11 files just for the tracks. But the other datasets are compressed in a single file each one.

In [5]:
list(tracks.columns)

['added_at',
 'added_by',
 'is_local',
 'playlist_id',
 'available_markets',
 'disc_number',
 'duration_ms',
 'explicit',
 'id',
 'name',
 'popularity',
 'track_number',
 'album_type',
 'album_available_markets',
 'album_id',
 'album_name',
 'album_release_date',
 'artists_ids',
 'artists_names',
 'album_artists_ids',
 'album_artists_names']

It's important to note that everything on Spotify have an ID. So when we have a song and know the ID of one playlist that contains it, we can use this playlist ID to search for the playlist data.

In the dataset, we see that a track have many features, which can be viewed as:
- **Playlist** features: information about the playlists, like when the song was added, who added it and the ID of the playlist.
- **Track** features: like `disc_number`, its duration, if its explicit, `id`, its name, popularity and its number.
- **Album** features, including artists of the album;
- **Artists** features: their IDs and names.

Many features have lists as values. It's because some of them have values which vary in length, like the `artists_ids`. We don't know if we will have 1, 2 or 32 artists. It will happen with other datasets too.

### Playlists dataset

In [6]:
playlists = pd.read_pickle('../data/sp_playlists.pkl')

In [7]:
playlists.sample()

,collaborative,description,id,name,primary_color,public,tracks,owner_id
3992,False,,3UuOn6KoHubwMvYuiPKx7E,Koh Lantana – Silent Moments,None,True,{'href': 'https://api.spotify.com/v1/playlists...,realjaco


Here we have basic information about a public playlist. One thing to note is that everything Spotipy returns is a JSON file, in the 'records' format. So in the column `tracks` we have a dict containing information about the tracks.

In [8]:
playlists.loc[0, 'tracks']

{'href': 'https://api.spotify.com/v1/playlists/0itjZK4e1qZzHL9fNInUJR/tracks',
 'total': 63}

It's little information, but it's not important. The tracks was gathered from the playlist using the ID of the playlist.

### Audio features dataset

Maybe only track information is not enough to make good recommendation systems. So we gather more information, this time more technician:

In [9]:
audio_features = pd.read_pickle('../data/sp_audio_features.pkl')

In [10]:
audio_features.sample()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
452117,0.462,0.727,7,-4.298,1,0.0383,0.212,0.0,0.192,0.475,138.806,45I1Wx82w8ArHSpV7dBp09,227440,3


Here we have the ID of the track and its features, like ` liveness` and `speechiness`. More information about this technical parte can be found on the [documentation of the Web API](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

Another thing to note is that some of `duration_ms` differs from the ones in the tracks database. Let's see:

In [19]:
my_id = None
while pd.isnull(my_id):
    my_id = tracks.sample().id.iloc[0] 

In [20]:
print(
    tracks[tracks.id == my_id].duration_ms.iloc[0],
    audio_features[audio_features.id == my_id].duration_ms.iloc[0]
)

256546.0 256547


### Artists dataset

In [22]:
artists = pd.read_pickle('../data/sp_artists.pkl')

In [23]:
artists.sample()

,followers,genres,id,name,popularity
65337,37284,"[alternative pop, alternative rock]",5LEUigTSXpwrtERRcSW1N4,Urge Overkill,51


Artists datasets are simple and autoexplanative. It's important to note `genres` features, because each cell is a list of genres. As already said, the only way to classify a song in a genre is with the genres of its artists.

## Exploration

Now he have the intuition and the knowlegde to explore the data.

### Tracks

#### Missing values and data types

Let's do the homework. The missing values:

In [24]:
tracks.dtypes

added_at                   datetime64[ns, UTC]
added_by                                object
is_local                                  bool
playlist_id                             object
available_markets                       object
disc_number                            float64
duration_ms                            float64
explicit                                object
id                                      object
name                                    object
popularity                             float64
track_number                           float64
album_type                              object
album_available_markets                 object
album_id                                object
album_name                              object
album_release_date                      object
artists_ids                             object
artists_names                           object
album_artists_ids                       object
album_artists_names                     object
dtype: object

dtypes seems to be OK, with exception of `explicit` (should be a bool) and `album_release_date` (should be a date). But they are extremely easy to deal with.

In [47]:
sum([pd.read_pickle(file).isnull().sum() for file in glob.glob('../data/sp_tracks_ready_*.pkl')])

added_at                       0
added_by                       0
is_local                       0
playlist_id                    0
available_markets            240
disc_number                  240
duration_ms                  240
explicit                     240
id                         39115
name                         240
popularity                   240
track_number                 240
album_type                 39115
album_available_markets      240
album_id                   39115
album_name                   240
album_release_date         39115
artists_ids                39115
artists_names                240
album_artists_ids          39115
album_artists_names        39115
dtype: int64

He have many missing values, in some specific tracks (the numbers coincide). Let's look into one of them:

In [25]:
# One of the 39115
tracks[pd.isnull(tracks.id)].sample()

,added_at,added_by,is_local,playlist_id,available_markets,disc_number,duration_ms,explicit,id,name,popularity,track_number,album_type,album_available_markets,album_id,album_name,album_release_date,artists_ids,artists_names,album_artists_ids,album_artists_names
60258,2016-05-14 18:44:15+00:00,marcosmoretto,True,00RyLvaDIOBJo2yfEq2Ewr,[],0.0,204000.0,False,None,Timber (feat. Ke$ha),0.0,0.0,None,[],None,Timber (feat. Ke$ha) - Single,None,NaN,[Pitbull],NaN,NaN


In [26]:
# One of the 240
tracks[pd.isnull(tracks.duration_ms)].sample()

,added_at,added_by,is_local,playlist_id,available_markets,disc_number,duration_ms,explicit,id,name,popularity,track_number,album_type,album_available_markets,album_id,album_name,album_release_date,artists_ids,artists_names,album_artists_ids,album_artists_names
51042,2020-07-31 17:18:56+00:00,katrin1994,False,7yh6OSV88noq3xosMtml7e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Number of tracks (it's huge)
sum([len(pd.read_pickle(file)) for file in glob.glob('../data/sp_tracks_ready_*.pkl')])

849397

In [28]:
print('Approx. {:.3f}% of the data has missing values.'.format((39115)/849397*100))

Approx. 4.605% of the data has missing values.


It's seems not to be a problem to drop these rows, because of the dropped data will be just a little. One problem is that it will affect the other datasets.

#### Features distribution

We will look into the distribution of the data, like the dates, the number and the categories. We will check for duplicity and will make a `describe`.

A track is dupplicated when it's the same track in the same playlist and was added at the same time.

In [34]:
duplicated = 0
for file in glob.glob('../data/sp_tracks_ready_*.pkl'):
    df = pd.read_pickle(file)
    duplicated += len(df[(df.id + df.playlist_id + df.added_at.apply(str)).duplicated()])
print(duplicated)

43781


We have many duplicated values. But the IDs where unified when dealling with audio features and artists.

We also found a playlist with the same song added 2048 times, with 1024 of them added at the same time!

In [35]:
tracks[(tracks.id == '3vllUzZChqpEaAat9brOYV') & (tracks.playlist_id == '3MPa29N5KpS3ZOk2B4PlUq') & (tracks.added_at == '2018-01-01 09:27:46+00:00')]

,added_at,added_by,is_local,playlist_id,available_markets,disc_number,duration_ms,explicit,id,name,popularity,track_number,album_type,album_available_markets,album_id,album_name,album_release_date,artists_ids,artists_names,album_artists_ids,album_artists_names
17191,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
17192,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
17193,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
17194,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
17195,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18210,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
18211,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
18212,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]
18213,2018-01-01 09:27:46+00:00,stabbedmeat,False,3MPa29N5KpS3ZOk2B4PlUq,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,205760.0,True,3vllUzZChqpEaAat9brOYV,FACK,41.0,2.0,compilation,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1zUY4PAFrNUOCeaEBrEHzh,Curtain Call: The Hits,2005-12-06,[7dGJo4pcD2V6oG8kP0tJRR],[Eminem],[7dGJo4pcD2V6oG8kP0tJRR],[Eminem]


Let's make a `describe`.

In [36]:
tracks.describe()

,disc_number,duration_ms,popularity,track_number
count,62628.000000,6.262800e+04,62628.000000,62628.000000
mean,1.000782,2.345405e+05,27.463116,5.116769
std,0.474441,9.027254e+04,26.100438,6.330573
min,0.000000,0.000000e+00,0.000000,0.000000
25%,1.000000,1.919060e+05,0.000000,1.000000
50%,1.000000,2.202930e+05,25.000000,3.000000
75%,1.000000,2.618800e+05,49.000000,7.000000
max,43.000000,4.681906e+06,100.000000,159.000000


There are tracks with duration 0:

In [37]:
tracks[tracks.duration_ms == 0]

,added_at,added_by,is_local,playlist_id,available_markets,disc_number,duration_ms,explicit,id,name,popularity,track_number,album_type,album_available_markets,album_id,album_name,album_release_date,artists_ids,artists_names,album_artists_ids,album_artists_names
39096,2010-12-08 20:23:57+00:00,,False,6KDJ5FumLzap9zJ2Ekaaxf,[],1.0,0.0,False,2h0PVoVGyPLYONgXDs6V6i,Too Late For Goodbye,0.0,3.0,album,[],4p6822xC4ni8qkEbLsHCjK,Burning The Day,2010-01-01,[04DUpHOyQqwbHFyvIhcGi3],[Randy Rogers Band],[04DUpHOyQqwbHFyvIhcGi3],[Randy Rogers Band]


In fact the song has 3min40s.

### Playlists

#### Missing values and data types

In [40]:
playlists.dtypes

collaborative      bool
description      object
id               object
name             object
primary_color    object
public             bool
tracks           object
owner_id         object
dtype: object

It seems OK.

In [41]:
playlists.isnull().sum()

collaborative        0
description          0
id                   0
name                 0
primary_color    11617
public               0
tracks               0
owner_id             0
dtype: int64

In [42]:
len(playlists)

11617

There are no `primary_colors`.

#### Features distribution

A track is dupplicated when it's the same track in the same playlist and was added at the same time.

In [45]:
playlists.duplicated('id').sum()

690

He have a lot of repeated playlists, but it was treated when dealing with the tracks!

### Audio features

#### Missing values and data types

In [49]:
audio_features.dtypes

danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
id                   object
duration_ms           int64
time_signature        int64
dtype: object

It seems OK.

In [50]:
audio_features.isnull().sum()

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
id                  0
duration_ms         0
time_signature      0
dtype: int64

Nice!

#### Features distribution

In [52]:
audio_features.duplicated('id').sum()

0

Nice too!

In [54]:
audio_features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,491595.000000,4.915950e+05,491595.000000
mean,0.555357,0.636276,5.281917,-8.746491,0.623934,0.096430,0.273871,0.202287,0.204890,0.470628,121.422468,2.426466e+05,3.905933
std,0.176397,0.248168,3.578772,4.880195,0.484397,0.143514,0.318675,0.331959,0.179179,0.257163,29.159548,1.205123e+05,0.435442
min,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.347000e+03,0.000000
25%,0.438000,0.466000,2.000000,-10.730000,0.000000,0.035500,0.010500,0.000001,0.096400,0.260000,99.875500,1.869330e+05,4.000000
50%,0.569000,0.679000,5.000000,-7.551000,1.000000,0.048300,0.116000,0.001000,0.129000,0.460000,121.030000,2.239600e+05,4.000000
75%,0.686000,0.843000,9.000000,-5.438000,1.000000,0.086100,0.494000,0.302000,0.263000,0.676000,138.700000,2.740400e+05,4.000000
max,0.993000,1.000000,11.000000,5.118000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,249.987000,5.925082e+06,5.000000


### Artists

#### Missing values and data types

In [55]:
artists.dtypes

followers      int64
genres        object
id            object
name          object
popularity     int64
dtype: object

In [56]:
artists.isnull().sum()

followers     0
genres        0
id            0
name          0
popularity    0
dtype: int64

It seems nice.

#### Features distribution

In [58]:
artists.duplicated('id').sum()

1

In [59]:
artists[artists.duplicated('id')]

,followers,genres,id,name,popularity
54492,46903,"[jazz funk, jazz fusion, jazz guitar]",3zX0EMvB00JzxnRi5EIICP,Mike Stern,36


In [60]:
artists[artists.id == '3zX0EMvB00JzxnRi5EIICP']

,followers,genres,id,name,popularity
1554,0,"[jazz funk, jazz fusion, jazz guitar]",3zX0EMvB00JzxnRi5EIICP,Mike Stern,36
54492,46903,"[jazz funk, jazz fusion, jazz guitar]",3zX0EMvB00JzxnRi5EIICP,Mike Stern,36


We have only one artist duplicated.

Let's make a `describe`.

In [61]:
artists.describe()

,followers,popularity
count,1.076220e+05,107622.000000
mean,7.451927e+04,28.218738
std,6.882583e+05,18.822269
min,0.000000e+00,0.000000
25%,1.950000e+02,12.000000
50%,1.628000e+03,28.000000
75%,1.212900e+04,42.000000
max,6.757670e+07,100.000000


The most followed artist:

In [63]:
artists[artists.followers == artists.followers.max()]

,followers,genres,id,name,popularity
84115,67576701,"[pop, uk pop]",6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,92
